In [1]:
import numpy as np
import glob
import re

from aicsimageio import AICSImage, readers
from pathlib import Path
from matplotlib import pyplot as plt

from blimp.preprocessing.illumination_correction import IlluminationCorrection

27-Jun-24 13:11:38 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


# Correct illumination

Search images of the correct channel in the input directories

In [2]:
images_dir_1 = Path("/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240304/240301_ARMC5KO_EU/20240304_180009_911/OME-TIFF-MIP/")
images_dir_2 = Path("/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240307/240304_ARMC5KO_EU/20240307_184329_444/OME-TIFF-MIP/")
images_dir_3 = Path("/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240229/240226_ARMC5KO_EU/20240229_172045_717/OME-TIFF-MIP/")
image_files_1 = glob.glob(str(images_dir_1 / "*.tiff"))
image_files_2 = glob.glob(str(images_dir_2 / "*.tiff"))
image_files_3 = glob.glob(str(images_dir_3 / "*.tiff"))

Bright dust in images can be detrimental for calculation of a standard deviation for illumination correction. We check that all images are within the same intensity range

In [3]:
def find_outlier_indices(image_files):
    n_channels = AICSImage(image_files[0],reader=readers.ome_tiff_reader.OmeTiffReader).dims.C
    image_std = np.stack(
        [[np.std(a=AICSImage(image_file,reader=readers.ome_tiff_reader.OmeTiffReader).get_image_data('YX',C=c)) for c in range(n_channels)] for image_file in image_files]
    )
    image_std_p25 = np.percentile(image_std,25,axis=0)
    image_std_p75 = np.percentile(image_std,75,axis=0)
    image_std_iqr = image_std_p75 - image_std_p25
    upper_limit = image_std_p75 + 1.5*image_std_iqr
    outlier = image_std > upper_limit
    outlier_indices = np.argwhere(outlier[:,0:2])[:,0]
    return outlier_indices, image_std

In [4]:
%%time
outlier_indices_1, image_std_1 = find_outlier_indices(image_files_1)
reference_image_files_1 = np.delete(image_files_1,outlier_indices_1)

CPU times: user 15min 42s, sys: 1min 45s, total: 17min 27s
Wall time: 23min 24s


In [5]:
%%time
outlier_indices_2, image_std_2 = find_outlier_indices(image_files_2)
reference_image_files_2 = np.delete(image_files_2,outlier_indices_2)

CPU times: user 27min 46s, sys: 3min 7s, total: 30min 53s
Wall time: 38min 20s


In [6]:
%%time
outlier_indices_3, image_std_3 = find_outlier_indices(image_files_3)
reference_image_files_3 = np.delete(image_files_3,outlier_indices_3)

CPU times: user 14min 49s, sys: 1min 45s, total: 16min 34s
Wall time: 18min 8s


In [7]:
image_files = image_files_1 + image_files_2 + image_files_3
reference_image_files = reference_image_files_1.tolist() + reference_image_files_2.tolist() + reference_image_files_3.tolist()

In [8]:
len(image_files)

3840

In [9]:
len(reference_image_files)

3465

Fit a ``blimp.IlluminationCorrection`` object

In [ ]:
illumination_correction = IlluminationCorrection(
    reference_images=reference_image_files,
    timelapse=False,
)
illumination_correction.plot()

# Save illumination correction object to disk

The ``blimp.IlluminationCorrection`` object can be persisted on disk to be later applied when analysing images

In [ ]:
illumcorr_filename = "/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240304/ILLUMCORR/illumination_correction.pkl"

In [ ]:
illumination_correction.save(Path(illumcorr_filename))

# Check results

IlluminationCorrection can be applied using the ``correct()`` method of the ``blimp.IlluminationCorrection`` object

In [ ]:
illumination_correction = IlluminationCorrection(from_file=illumcorr_filename)

In [ ]:
illumination_correction.plot()

In [ ]:
raw = AICSImage(image_files[705], reader=readers.ome_tiff_reader.OmeTiffReader)
corrected = illumination_correction.correct(raw)

In [ ]:
illumination_correction.mean_mean_image

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
im = axes[0].imshow(raw.get_image_data('YX',C=0,T=0), vmin=0, vmax=1000)
fig.colorbar(im, ax=axes[0])
axes[0].set_title("Original")
im = axes[1].imshow(corrected.get_image_data('YX',C=0,T=0), vmin=0, vmax=1000)
fig.colorbar(im, ax=axes[1])
axes[1].set_title("Corrected")
fig.tight_layout()

Will have to crop part of these images (from say 500: on the X-axis)